In [44]:
from kalman import *
from animation import Animator
import matplotlib.pyplot as plt
%matplotlib tk

Konstanty. g je gravitační zrychlení a R je poloměr Země.

In [15]:
R = 6371000
g = 9.81

Nejprve ukážeme model, ve kterém působí pouze gravitační síla.

In [20]:
class MissileModel1(ModelProjekt):

    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0],
                          [0, 1, 0, self.D],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0],
                          [0, 1, 0, 0]]))

    def Qk(self, k):
        return 0.01 * np.eye(4)

    def Rk(self, k):
        return 0.001 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']
        return np.array([0, -g * (1 - (2 * y / R))])

In [24]:
class FriendlyMissile1(Model):

    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0],
                          [0, 1, 0, self.D],
                          [0, 0, 1, 0],
                          [0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0],
                          [0, 1, 0, 0]]))

    def Qk(self, k):
        return 0.01 * np.eye(4)

    def Rk(self, k):
        return 0.001 * np.eye(2)

    def Uk(self, k, **kwargs):
        return np.array([0, 0])


Nastavíme délku časového kroku.

In [25]:
Missile1 = MissileModel1()
Missile1.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

Vytvoříme instanci třídy Simulation. Ta nám bude simulovat pohyb bomby, protibombového systému a samotnou predikci polohy bomby

In [26]:
S = Simulation(Missile1, Friendly, np.array([0,0,0,0]),np.diag([1e6, 1e6, 1e6, 1e6]), np.array([0, 15000, 555, 0]),  np.array([30000, 0, 0, 0]))
anim = Animator(S, 1000)
anim.start_animation()


Na bombu bude působit vítr. Navíc budeme brát v potaz odpor vzduchu ve směru os x a y.

In [64]:
wind = 25

class MissileModel2(Model):

    kx = 10
    ky = 10
    m = 2000


    def Ak(self, k, **kwargs):
        return (np.array([[1, 0, self.D, 0, 0],
                          [0, 1, 0, self.D, 0],
                          [0, 0, 1 - (self.kx/self.m)*self.D, 0, (self.kx/self.m)*self.D],
                          [0, 0, 0, 1 - (self.ky/self.m)*self.D, 0],
                          [0, 0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D],
                          [0, 0]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0]]))

    def Qk(self, k):
        Q = 0.01 * np.eye(5)
        Q[4,4] = 0.5
        return Q

    def Rk(self, k):
        return 400 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']
        return np.array([0, -g * (1 - (2 * y / R))])

In [65]:
Missile2 = MissileModel2()
Missile2.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

S = Simulation(enemy_missile=Missile2, friendly_missile=Friendly, mu0=np.array([-10000, 20000, 200, 0, 0]),
               sigma0=np.diag([1e6, 1e6, 1e6, 1e6, 1e6]), system_x0=np.array([0, 8000, 555, 0, wind]),
               fr_system_x0=np.array([20000, 0, 0, 0]))
anim = Animator(S, 10000)
anim.start_animation()

In [66]:
plt.figure(figsize=(20, 10))
plt.subplot(221)
plt.plot(S.system.tracex[:, 0], S.system.tracex[:, 1])
plt.xlabel('$x$')
plt.ylabel('$y$')

plt.subplot(223)
plt.plot(S.system.tracet, S.system.tracex[:, 2])
plt.ylabel('$v_x$')
plt.xlabel('$t$')

plt.subplot(224)
plt.plot(S.system.tracet, S.system.tracex[:, 3])
plt.ylabel('$v_y$')
plt.xlabel('$t$')

plt.subplot(222)
plt.plot(S.system.tracet, S.system.tracex[:, 4])
plt.ylabel('$u$')
plt.xlabel('$t$')

Text(0.5, 0, '$t$')

Představme si situaci, kdy chceme zvýšit dolet bomby. Existují tzv. klouzavé bomby, kterým se po vypuštění z bombardéru roztáhnou křídla. Na křídla působí vztlaková síla a odpor vzduchu ve směru osy y je větší. Ukážeme si dva modely a v obou budeme měnit koeficient odporu vzduchu. Křídla bomby jsou ve statické poloze (nijak se nenaklání) a tudíž koeficient odporu se mění v závislosti na úhlu, které svírají mezi sebou složky rychlost vx a vy. Čím větší úhel mezi nimi (to znamená, že bomba klesá strměji), tím menší bude odpor. V každé iteraci vzpočteme koeficient odporu vzduchu.

In [83]:
wind = 2
koeficient_y = 15 # a 15
bomb_x = 0 #  pozice bomby na zacatku v ose x
class MissileModel3(Model):

    kx = 10
    ky = 0
    m = 2000


    def Ak(self, k, **kwargs):
        vy = kwargs['vy']
        vx = kwargs['vx']
        # print(ky)
        self.ky = 50 + (1/(np.arctan(abs(vy+0.001) / abs(vx+0.001))))*koeficient_y

        return (np.array([[1, 0, self.D, 0, 0],
                          [0, 1, 0, self.D, 0],
                          [0, 0, 1 - (self.kx/self.m)*self.D, 0, (self.kx/self.m)*self.D],
                          [0, 0, 0, 1 - (self.ky/self.m)*self.D, 0],
                          [0, 0, 0, 0, 1]]))

    def Bk(self, k):
        return (np.array([[0, 0],
                          [0, 0],
                          [self.D, 0],
                          [0, self.D],
                          [0, 0]]))

    def Hk(self, k):
        return (np.array([[1, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0]]))

    def Qk(self, k):
        Q = 0.01 * np.eye(5)
        Q[4,4] = 0.5
        return Q

    def Rk(self, k):
        return 400 * np.eye(2)

    def Uk(self, k, **kwargs):
        y = kwargs['y']


        return np.array([0, -g * (1 - (2 * y / R))])


In [84]:
Missile3 = MissileModel3()
Missile3.set_D(0.1)
Friendly = FriendlyMissile1()
Friendly.set_D(0.1)

In [85]:
S = Simulation(enemy_missile=Missile3, friendly_missile=Friendly, mu0=np.array([-10000, 20000, 200, 0, 0]),
               sigma0=np.diag([1e6, 1e6, 1e6, 1e6, 1e6]), system_x0=np.array([bomb_x, 8000, 555, 0, wind]),
               fr_system_x0=np.array([26000, 0, 0, 0]))

anim = Animator(S, 10000)
anim.start_animation()

In [82]:
plt.figure(figsize=(20, 10))
plt.subplot(221)
plt.plot(S.system.tracex[:, 0], S.system.tracex[:, 1])
plt.xlabel('$x$')
plt.ylabel('$y$')

plt.subplot(223)
plt.plot(S.system.tracet, S.system.tracex[:, 2])
plt.ylabel('$v_x$')
plt.xlabel('$t$')

plt.subplot(224)
plt.plot(S.system.tracet, S.system.tracex[:, 3])
plt.ylabel('$v_y$')
plt.xlabel('$t$')

plt.subplot(222)
plt.plot(S.system.tracet, S.system.tracex[:, 4])
plt.ylabel('$u$')
plt.xlabel('$t$')

TypeError: 'NoneType' object is not subscriptable